# Delta Architecture: Beyond Lambda Architecture

<img src="https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-logo-whitebackground.png" width=200/>

This is a companion notebook to provide a Delta Lake example against the Lending Club data.
* This notebook has been tested with *DBR 5.4 ML Beta, Python 3*

## The Data

The data used is public data from Lending Club. It includes all funded loans from 2012 to 2017. Each loan includes applicant information provided by the applicant as well as the current loan status (Current, Late, Fully Paid, etc.) and latest payment information. For a full view of the data please view the data dictionary available [here](https://resources.lendingclub.com/LCDataDictionary.xlsx).


![Loan_Data](https://preview.ibb.co/d3tQ4R/Screen_Shot_2018_02_02_at_11_21_51_PM.png)

https://www.kaggle.com/wendykan/lending-club-loan-data

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Delta Lake

Optimization Layer a top blob storage for Reliability (i.e. ACID compliance) and Low Latency of Streaming + Batch data pipelines.

## Import Data and create pre-Delta Lake Table
* This will create a lot of small Parquet files emulating the typical small file problem that occurs with streaming or highly transactional data

In [5]:
# -----------------------------------------------
# Uncomment and run if this folder does not exist
# -----------------------------------------------
# Configure location of loanstats_2012_2017.parquet
lspq_path = "/databricks-datasets/samples/lending_club/parquet/"

# Read loanstats_2012_2017.parquet
data = spark.read.parquet(lspq_path)

# Reduce the amount of data (to run on DBCE)
(loan_stats, loan_stats_rest) = data.randomSplit([0.01, 0.99], seed=123)

# Select only the columns needed
loan_stats = loan_stats.select("addr_state", "loan_status")

# Create loan by state
loan_by_state = loan_stats.groupBy("addr_state").count()

# Create table
loan_by_state.createOrReplaceTempView("loan_by_state")

# Display loans by state
display(loan_by_state)

addr_state,count
AZ,336
SC,172
LA,171
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
WY,31


## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Easily Convert Parquet to Delta Lake format
With Delta Lake, you can easily transform your Parquet data into Delta Lake format.

In [7]:
# Configure Delta Lake Silver Path
DELTALAKE_SILVER_PATH = "/ml/loan_by_state_delta"

# Remove folder if it exists
dbutils.fs.rm(DELTALAKE_SILVER_PATH, recurse=True)

Out[2]: True

In [8]:
%sql 
-- Current example is creating a new table instead of in-place import so will need to change this code
DROP TABLE IF EXISTS loan_by_state_delta;

CREATE TABLE loan_by_state_delta
USING delta
LOCATION '/ml/loan_by_state_delta'
AS SELECT * FROM loan_by_state;

-- View Delta Lake table
SELECT * FROM loan_by_state_delta

addr_state,count
OK,136
DC,40
WI,200
ID,15
MD,369
GA,503
AR,104
NC,396
AK,32
UT,77


In [9]:
%sql 
DESCRIBE DETAIL delta.`/ml/loan_by_state_delta`

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion
delta,53667fe3-935b-4453-b76b-c2404fbba01f,null,null,dbfs:/ml/loan_by_state_delta,2020-03-05T17:15:28.921+0000,2020-03-05T17:15:38.000+0000,List(),46,30507,Map(),1,2


## Stop the notebook before the streaming cell, in case of a "run all"

In [11]:
dbutils.notebook.exit("stop") 

stop

In [12]:
%sh 
ls -lt /dbfs/ml/loan_by_state_delta/

total 50
-rwxrwxrwx 1 root root 668 Mar 5 17:15 part-00091-c2bb4996-b42e-4c00-9757-c2d724517d31-c000.snappy.parquet
-rwxrwxrwx 1 root root 669 Mar 5 17:15 part-00108-529770d7-aeb9-4901-89ba-4843f4ca02fe-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00110-d2e09790-a258-47f3-8f28-524514327783-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00115-c6682306-9b27-4702-854b-602af3760812-c000.snappy.parquet
-rwxrwxrwx 1 root root 669 Mar 5 17:15 part-00116-947799db-c4ac-4cce-b7b6-896a825d7b21-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00130-d1a7abf1-ed35-4898-b14f-0bd9db368e98-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00151-2e69af8b-661a-43a5-9d7e-e6f08a1020bc-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00154-fa41b91b-f58a-4189-8e4f-698244d33554-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00165-619a3128-adb9-43f8-87c9-b11c0446a10d-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00167-bb9ff08e-26e4-4008-963e-a095b19f2082-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00169-14366efa-8bcc-42f4-a738-90346f411a57-c000.snappy.parquet
-rwxrwxrwx 1 root root 668 Mar 5 17:15 part-00175-e93f72de-d798-4f17-8e2d-de677aa3ec26-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00179-a0de01dd-1b55-4493-9d80-a8e8798a9e17-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00183-ea3ad85b-236b-4f1e-82d8-ba91ff7403cc-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00185-563301e8-2e99-4b41-b0e7-1a8f3c9bb8e1-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00195-3eded3cd-2837-4c2f-bbd7-ff46af959af4-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00199-0ce57f67-d60b-4e3b-941f-10f73403c43c-c000.snappy.parquet
-rwxrwxrwx 1 root root 373 Mar 5 17:15 part-00000-f779b3e4-d600-4bb0-aba8-62492a55beeb-c000.snappy.parquet
-rwxrwxrwx 1 root root 668 Mar 5 17:15 part-00004-15bf774a-d597-4518-ac11-c173591df082-c000.snappy.parquet
-rwxrwxrwx 1 root root 669 Mar 5 17:15 part-00009-8966ec3c-3485-4c6b-b0da-02b94846f00b-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00010-9262e3b0-3c1d-42e4-8164-22d14cd27297-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00016-eb0b50c9-4080-49ff-a0e1-ba7d6fba3095-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00021-43fd74b7-75d3-48a6-8221-4dd84cf962af-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00030-6ea8e15b-544f-41ca-910d-6dc42dcba124-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00042-adda56a5-f39f-4da2-a0cc-98b385ae3add-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00047-48a65644-ce43-40f1-9a1f-f8e05b1048b1-c000.snappy.parquet
-rwxrwxrwx 1 root root 668 Mar 5 17:15 part-00049-09ca4c03-df6f-4660-a7d4-e5613a9974df-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00051-4a7b555b-4738-4c05-a590-eb56e5dae5f3-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00054-783ce128-13ec-4bac-be9e-d027aeb474ac-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00055-3c27b01a-10bf-4b32-a621-7cac6feb7713-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00059-ec3acbab-cdc6-44e4-a9d9-eb3a3cd6e5cd-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00060-308285b0-34c0-489a-a9ee-27947dd50f67-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00065-c2c9db71-cede-4723-9567-f1570937ed85-c000.snappy.parquet
-rwxrwxrwx 1 root root 667 Mar 5 17:15 part-00066-cbd2eb2e-0283-40b2-b8e8-e0a5ddc0d632-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00067-7be280bb-647a-4160-85b6-de0d384524c7-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00070-e50d0f46-1d97-46b0-a7d6-98045d0ea453-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5 17:15 part-00071-6ee35916-70ac-4b3c-b745-8eff0b4cdb1d-c000.snappy.parquet
-rwxrwxrwx 1 root root 670 Mar 5

In [13]:
%sh 
ls -lt /dbfs/ml/loan_by_state_delta/_delta_log/

total 17
-rwxrwxrwx 1 root root 91 Mar 5 17:15 00000000000000000000.crc
-rwxrwxrwx 1 root root 16702 Mar 5 17:15 00000000000000000000.json

## ![Delta Lake Tiny Logo](https://pages.databricks.com/rs/094-YMS-629/images/delta-lake-tiny-logo.png) Unified Batch and Streaming Source and Sink

These cells showcase streaming and batch concurrent queries (inserts and reads)
* This notebook will run an `INSERT` every 10s against our `loan_stats_delta` table
* We will run two streaming queries concurrently against this data
* Note, you can also use `writeStream` but this version is easier to run in DBCE

In [15]:
# Read the insertion of data
spark.sql("set spark.sql.shuffle.partitions = 1")
spark.sql("set spark.databricks.delta.snapshotPartitions = 1")

loan_by_state_readStream = spark.readStream.format("delta").load(DELTALAKE_SILVER_PATH)
loan_by_state_readStream.createOrReplaceTempView("loan_by_state_readStream")

In [16]:
%sql
select addr_state, sum(`count`) as loans from loan_by_state_readStream group by addr_state

addr_state,loans
MI,351
IL,580
WY,31
AR,104
OR,182
SC,172
AZ,336
IA,3285
MN,261
SD,30


**Wait** until the stream is up and running before executing the code below

In [18]:
import random
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *


def random_checkpoint_dir(): 
  return "/tmp/loan_by_state_delta/chkpt/%s" % str(random.randint(0, 10000))

states = ["IA", "WA"]

@udf(returnType=StringType())
def random_state():
  return str(random.choice(states))

# Function to start a streaming query with a stream of randomly generated data and append to the parquet table
def generate_and_append_data_stream(table_format, table_path):

  stream_data = (spark.readStream.format("rate").option("rowsPerSecond", 1).load() 
    .withColumn("addr_state", random_state())
    #.withColumn("count", lit(45))
    .withColumn("count", lit(45).cast("long"))                 
    .select("addr_state", "count")
  )

  query = (stream_data.writeStream 
    .format(table_format) 
    .option("checkpointLocation", random_checkpoint_dir()) 
    .trigger(processingTime = "10 seconds") 
    .start(table_path))

  return query

# Function to stop all streaming queries 
def stop_all_streams():
  # Stop all the streams
  print("Stopping all streams")
  for s in spark.streams.active:
    s.stop()
  print("Stopped all streams")
  print("Deleting checkpoints")  
  dbutils.fs.rm("/tmp/loan_by_state_delta/chkpt/", True)
  print("Deleted checkpoints")

In [19]:
stream_query_2 = generate_and_append_data_stream(table_format = "delta", table_path = DELTALAKE_SILVER_PATH)

In [20]:
stream_query_3 = generate_and_append_data_stream(table_format = "delta", table_path = DELTALAKE_SILVER_PATH)

In [21]:
%sql
select addr_state, sum(`count`) from loan_by_state_delta group by addr_state

addr_state,sum(count)
WA,4476
IA,3510
MN,261
NJ,540
DC,40
OR,182
VA,424
RI,65
NH,71
MI,351


In [22]:
stop_all_streams()

Stopping all streams
Stopped all streams
Deleting checkpoints
Deleted checkpoints

**Note**: Once the previous cell is finished and the state of Iowa is fully populated in the map (in cell 14), click *Cancel* in Cell 14 to stop the `readStream`.

In [24]:
%sh 
ls -lt /dbfs/ml/loan_by_state_delta/_delta_log/

total 149
-rwxrwxrwx 1 root root 93 Mar 5 17:20 00000000000000000021.crc
-rwxrwxrwx 1 root root 3962 Mar 5 17:20 00000000000000000022.json
-rwxrwxrwx 1 root root 3960 Mar 5 17:20 00000000000000000021.json
-rwxrwxrwx 1 root root 24196 Mar 5 17:20 00000000000000000020.checkpoint.parquet
-rwxrwxrwx 1 root root 26 Mar 5 17:20 _last_checkpoint
-rwxrwxrwx 1 root root 3962 Mar 5 17:20 00000000000000000020.json
-rwxrwxrwx 1 root root 93 Mar 5 17:20 00000000000000000019.crc
-rwxrwxrwx 1 root root 3960 Mar 5 17:20 00000000000000000019.json
-rwxrwxrwx 1 root root 93 Mar 5 17:20 00000000000000000018.crc
-rwxrwxrwx 1 root root 3962 Mar 5 17:20 00000000000000000018.json
-rwxrwxrwx 1 root root 93 Mar 5 17:20 00000000000000000017.crc
-rwxrwxrwx 1 root root 3960 Mar 5 17:20 00000000000000000017.json
-rwxrwxrwx 1 root root 93 Mar 5 17:20 00000000000000000015.crc
-rwxrwxrwx 1 root root 3962 Mar 5 17:20 00000000000000000016.json
-rwxrwxrwx 1 root root 3960 Mar 5 17:20 00000000000000000015.json
-rwxrwxrwx 1 root root 92 Mar 5 17:19 00000000000000000013.crc
-rwxrwxrwx 1 root root 3960 Mar 5 17:19 00000000000000000014.json
-rwxrwxrwx 1 root root 3960 Mar 5 17:19 00000000000000000013.json
-rwxrwxrwx 1 root root 3960 Mar 5 17:19 00000000000000000012.json
-rwxrwxrwx 1 root root 92 Mar 5 17:19 00000000000000000011.crc
-rwxrwxrwx 1 root root 3960 Mar 5 17:19 00000000000000000011.json
-rwxrwxrwx 1 root root 20176 Mar 5 17:19 00000000000000000010.checkpoint.parquet
-rwxrwxrwx 1 root root 92 Mar 5 17:19 00000000000000000009.crc
-rwxrwxrwx 1 root root 3959 Mar 5 17:19 00000000000000000010.json
-rwxrwxrwx 1 root root 2575 Mar 5 17:19 00000000000000000009.json
-rwxrwxrwx 1 root root 92 Mar 5 17:19 00000000000000000008.crc
-rwxrwxrwx 1 root root 749 Mar 5 17:19 00000000000000000008.json
-rwxrwxrwx 1 root root 92 Mar 5 17:19 00000000000000000007.crc
-rwxrwxrwx 1 root root 3959 Mar 5 17:19 00000000000000000007.json
-rwxrwxrwx 1 root root 91 Mar 5 17:19 00000000000000000006.crc
-rwxrwxrwx 1 root root 3959 Mar 5 17:19 00000000000000000006.json
-rwxrwxrwx 1 root root 91 Mar 5 17:19 00000000000000000005.crc
-rwxrwxrwx 1 root root 3959 Mar 5 17:19 00000000000000000005.json
-rwxrwxrwx 1 root root 91 Mar 5 17:18 00000000000000000004.crc
-rwxrwxrwx 1 root root 3959 Mar 5 17:18 00000000000000000004.json
-rwxrwxrwx 1 root root 91 Mar 5 17:18 00000000000000000003.crc
-rwxrwxrwx 1 root root 3959 Mar 5 17:18 00000000000000000003.json
-rwxrwxrwx 1 root root 91 Mar 5 17:18 00000000000000000002.crc
-rwxrwxrwx 1 root root 1883 Mar 5 17:18 00000000000000000002.json
-rwxrwxrwx 1 root root 91 Mar 5 17:18 00000000000000000001.crc
-rwxrwxrwx 1 root root 749 Mar 5 17:18 00000000000000000001.json
-rwxrwxrwx 1 root root 91 Mar 5 17:15 00000000000000000000.crc
-rwxrwxrwx 1 root root 16702 Mar 5 17:15 00000000000000000000.json

In [25]:
%sh 
head /dbfs/ml/loan_by_state_delta/_delta_log/00000000000000000014.json

{"commitInfo":{"timestamp":1583428790900,"userId":"1519579297082103","userName":"denny.lee@databricks.com","operation":"STREAMING UPDATE","operationParameters":{"outputMode":"Append","queryId":"958fd73b-aa2f-4f35-a6d3-4732a6b5e727","epochId":"3"},"notebook":{"notebookId":"401867962045400"},"clusterId":"0227-055352-aurae397","readVersion":12,"isolationLevel":"WriteSerializable","isBlindAppend":true}}
{"txn":{"appId":"958fd73b-aa2f-4f35-a6d3-4732a6b5e727","version":3,"lastUpdated":1583428790899}}
{"add":{"path":"part-00000-35fccb12-446d-4532-b6c8-a6750ce9baff-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00001-2073414d-d189-4389-92dd-0cbcae0a516b-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00002-40da0708-c15a-40d9-acfd-4a777d1dc9da-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00003-d31c47a2-7a95-4985-bfd9-51fb578f180f-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00004-afc240a8-02b5-403d-83de-849a1bdae0ea-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00005-5b46c588-0a90-4cf0-a623-3e580d7df2ad-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00006-c1dc3e9e-12e6-4b37-bd47-82bfe2c2c158-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00007-5ed05e9f-9590-4c17-b534-6ed8768566d4-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}

In [26]:
%sh 
head /dbfs/ml/loan_by_state_delta/_delta_log/00000000000000000013.json

{"commitInfo":{"timestamp":1583428790659,"userId":"1519579297082103","userName":"denny.lee@databricks.com","operation":"STREAMING UPDATE","operationParameters":{"outputMode":"Append","queryId":"2e3ffd54-40c5-41dc-ac7a-0a0bfd506672","epochId":"9"},"notebook":{"notebookId":"401867962045400"},"clusterId":"0227-055352-aurae397","readVersion":12,"isolationLevel":"WriteSerializable","isBlindAppend":true}}
{"txn":{"appId":"2e3ffd54-40c5-41dc-ac7a-0a0bfd506672","version":9,"lastUpdated":1583428790659}}
{"add":{"path":"part-00000-bef22015-d93b-40ce-b02d-668596c205e6-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00001-9159cc08-0246-4dcf-84c4-fd233d474642-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00002-145da694-d649-4bfa-82c8-a266e1c6049c-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00003-6a8ddcbe-b8b7-43f1-bc79-dd55a4b1b7f1-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00004-6093f5a6-d17e-451f-9522-e029aa5d63b7-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00005-dc3a0a84-fab0-492c-ba05-6bf449d6092f-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00006-ccf7abc7-d0a9-4345-9556-aa4dfa5004f4-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"IA\",\"count\":45},\"maxValues\":{\"addr_state\":\"IA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}
{"add":{"path":"part-00007-7ef8c938-0eba-4a5c-8a12-cd2fd7fa4a29-c000.snappy.parquet","partitionValues":{},"size":670,"modificationTime":1583428790000,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"addr_state\":\"WA\",\"count\":45},\"maxValues\":{\"addr_state\":\"WA\",\"count\":45},\"nullCount\":{\"addr_state\":0,\"count\":0}}"}}